# Overfitting in CNN
---

Applying **Layer Normalization** and **Regularization**

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
EPOCHS = 100

In [3]:
# Define the network architecture.
# a sort of VGGNet 16
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1_1 = tf.keras.layers.Conv2D(16, (3, 3), padding = 'same',
                                              activation = 'relu')
        self.conv1_2 = tf.keras.layers.Conv2D(16, (3, 3), padding = 'same',
                                              activation = 'relu')
        self.pool1 = tf.keras.layers.MaxPool2D((2, 2))
        
        self.conv2_1 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'same',
                                              activation = 'relu')
        self.conv2_2 = tf.keras.layers.Conv2D(32, (3, 3), padding = 'same',
                                              activation = 'relu')
        self.pool2 = tf.keras.layers.MaxPool2D((2, 2))
        
        self.conv3_1 = tf.keras.layers.Conv2D(64, (3, 3), padding = 'same',
                                              activation = 'relu')
        self.conv3_2 = tf.keras.layers.Conv2D(64, (3, 3), padding = 'same',
                                              activation = 'relu')
        
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(1024, activation = 'relu')
        self.dense2 = tf.keras.layers.Dense(10, activation = 'softmax')
        
    def call(self, x, training = False, mask = None):
        x = self.conv1_1(x)
        x = self.conv1_2(x)
        x = self.pool1(x)
        
        x = self.conv2_1(x)
        x = self.conv2_2(x)
        x = self.pool2(x)
        
        x = self.conv3_1(x)
        x = self.conv3_2(x)
        
        x = self.flatten(x)
        x = self.dense1(x)
        
        return self.dense2(x)

In [4]:
# loading data
cifar10 = tf.keras.datasets.cifar10 # 32 X 32 X 3 video (10 classes)

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32).prefetch(1024)
test_ds = tf.data.Dataset.from_tensor_slices(
    (x_test, y_test)).batch(32).prefetch(1024)

In [5]:
# training
model = MyModel()
model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])
model.fit(train_ds, validation_data = test_ds, epochs = EPOCHS)

Epoch 1/100
1563/1563 [==============================] - 16s 8ms/step - loss: 1.4349 - accuracy: 0.4754 - val_loss: 1.1130 - val_accuracy: 0.5980
Epoch 2/100
1563/1563 [==============================] - 12s 8ms/step - loss: 0.9590 - accuracy: 0.6611 - val_loss: 0.9133 - val_accuracy: 0.6772
Epoch 3/100
1563/1563 [==============================] - 12s 8ms/step - loss: 0.7559 - accuracy: 0.7324 - val_loss: 0.9043 - val_accuracy: 0.6944
Epoch 4/100
1563/1563 [==============================] - 13s 8ms/step - loss: 0.5837 - accuracy: 0.7939 - val_loss: 0.8210 - val_accuracy: 0.7308
Epoch 5/100
1563/1563 [==============================] - 13s 8ms/step - loss: 0.4055 - accuracy: 0.8569 - val_loss: 1.0431 - val_accuracy: 0.6996
Epoch 6/100
1563/1563 [==============================] - 13s 9ms/step - loss: 0.2648 - accuracy: 0.9068 - val_loss: 1.2438 - val_accuracy: 0.7092
Epoch 7/100
1563/1563 [==============================] - 13s 8ms/step - loss: 0.1764 - accuracy: 0.9392 - val_loss: 1.2650 -

Epoch 57/100
1563/1563 [==============================] - 13s 8ms/step - loss: 0.0651 - accuracy: 0.9851 - val_loss: 4.5613 - val_accuracy: 0.6831
Epoch 58/100
1563/1563 [==============================] - 13s 8ms/step - loss: 0.0825 - accuracy: 0.9816 - val_loss: 4.1591 - val_accuracy: 0.6861
Epoch 59/100
1563/1563 [==============================] - 13s 8ms/step - loss: 0.0700 - accuracy: 0.9855 - val_loss: 4.1197 - val_accuracy: 0.7059
Epoch 60/100
1563/1563 [==============================] - 13s 8ms/step - loss: 0.0763 - accuracy: 0.9840 - val_loss: 4.0153 - val_accuracy: 0.6899
Epoch 61/100
1563/1563 [==============================] - 12s 8ms/step - loss: 0.0738 - accuracy: 0.9833 - val_loss: 4.1916 - val_accuracy: 0.6968
Epoch 62/100
1563/1563 [==============================] - 12s 8ms/step - loss: 0.0801 - accuracy: 0.9829 - val_loss: 4.3168 - val_accuracy: 0.6896
Epoch 63/100
1563/1563 [==============================] - 13s 8ms/step - loss: 0.0780 - accuracy: 0.9835 - val_loss: 3

In [6]:
# For now, the gap between the training performance
# and the validation performace is too large. -> Overfitting!

# Layer Normalization ('use_bias' = False due to Layer Normalization
#                                         Recall 'fuse'!)
class ConvLNReLUBlock(tf.keras.Model):
    def __init__(self, num_filters, kernel_size):
        super(ConvLNReLUBlock, self).__init__()
        self.conv = tf.keras.layers.Conv2D(num_filters, kernel_size,
                                           padding = 'same', use_bias = False)
        self.ln = tf.keras.layers.LayerNormalization()
        
    def call(self, x, training = False, mask = None):
        x = self.conv(x)
        # Layer Normalization has to do with layer not batch.
        # Thus, we do not have to insert 'training flag' into
        # the self.ln().
        x = self.ln(x)
        
        return tf.nn.relu(x)

In [7]:
# applying Layer Normalization and Regularization (Kernel Regularization)
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1_1 = ConvLNReLUBlock(16, (3, 3))
        self.conv1_2 = ConvLNReLUBlock(16, (3, 3))
        self.pool1 = tf.keras.layers.MaxPool2D((2, 2))
        
        self.conv2_1 = ConvLNReLUBlock(32, (3, 3))
        self.conv2_2 = ConvLNReLUBlock(32, (3, 3))
        self.pool2 = tf.keras.layers.MaxPool2D((2, 2))
        
        self.conv3_1 = ConvLNReLUBlock(64, (3, 3))
        self.conv3_2 = ConvLNReLUBlock(64, (3, 3))
        
        self.flatten = tf.keras.layers.Flatten()
        # kernel_regularization: kernel = filter (weights)
        # l2: L2-Regularization (In this case, lambda = 0.01.)
        self.dense1 = tf.keras.layers.Dense(1024, activation = 'relu',
                                            kernel_regularizer = tf.keras.regularizers.l2(0.01))
        self.dense2 = tf.keras.layers.Dense(10, activation = 'softmax',
                                            kernel_regularizer = tf.keras.regularizers.l2(0.01))
        
    def call(self, x, training = False, mask = None):
        x = self.conv1_1(x)
        x = self.conv1_2(x)
        x = self.pool1(x)
        
        x = self.conv2_1(x)
        x = self.conv2_2(x)
        x = self.pool2(x)
        
        x = self.conv3_1(x)
        x = self.conv3_2(x)
        
        x = self.flatten(x)
        x = self.dense1(x)
        
        return self.dense2(x)

In [8]:
# training
model2 = MyModel()
model2.compile(optimizer = 'adam',
               loss = 'sparse_categorical_crossentropy',
               metrics = ['accuracy'])
model2.fit(train_ds, validation_data = test_ds, epochs = EPOCHS)

# improved validation performance

Epoch 1/100
1563/1563 [==============================] - 37s 23ms/step - loss: 2.1760 - accuracy: 0.4070 - val_loss: 1.3853 - val_accuracy: 0.5374
Epoch 2/100
1563/1563 [==============================] - 34s 22ms/step - loss: 1.2852 - accuracy: 0.5889 - val_loss: 1.1691 - val_accuracy: 0.6378
Epoch 3/100
1563/1563 [==============================] - 34s 22ms/step - loss: 1.1085 - accuracy: 0.6525 - val_loss: 1.0425 - val_accuracy: 0.6849
Epoch 4/100
1563/1563 [==============================] - 32s 21ms/step - loss: 1.0132 - accuracy: 0.6864 - val_loss: 0.9727 - val_accuracy: 0.7126
Epoch 5/100
1563/1563 [==============================] - 32s 21ms/step - loss: 0.9460 - accuracy: 0.7117 - val_loss: 0.9531 - val_accuracy: 0.7158
Epoch 6/100
1563/1563 [==============================] - 32s 21ms/step - loss: 0.8918 - accuracy: 0.7303 - val_loss: 0.9544 - val_accuracy: 0.7065
Epoch 7/100
1563/1563 [==============================] - 32s 21ms/step - loss: 0.8454 - accuracy: 0.7451 - val_loss: 0

1563/1563 [==============================] - 34s 22ms/step - loss: 0.2821 - accuracy: 0.9479 - val_loss: 0.9685 - val_accuracy: 0.7806
Epoch 57/100
1563/1563 [==============================] - 34s 22ms/step - loss: 0.2757 - accuracy: 0.9503 - val_loss: 0.9904 - val_accuracy: 0.7770
Epoch 58/100
1563/1563 [==============================] - 34s 22ms/step - loss: 0.2710 - accuracy: 0.9511 - val_loss: 1.0146 - val_accuracy: 0.7688
Epoch 59/100
1563/1563 [==============================] - 33s 21ms/step - loss: 0.2702 - accuracy: 0.9520 - val_loss: 1.0789 - val_accuracy: 0.7655
Epoch 60/100
1563/1563 [==============================] - 32s 20ms/step - loss: 0.2630 - accuracy: 0.9540 - val_loss: 1.0397 - val_accuracy: 0.7789
Epoch 61/100
1563/1563 [==============================] - 33s 21ms/step - loss: 0.2678 - accuracy: 0.9517 - val_loss: 1.0115 - val_accuracy: 0.7730
Epoch 62/100
1563/1563 [==============================] - 33s 21ms/step - loss: 0.2636 - accuracy: 0.9536 - val_loss: 1.0006 

# Underfitting
---

With restricted computing resources, maximize the model's performance. <br>$\rightarrow$ Naturally, the model is susceptible to **underfitting problems**.

In [10]:
class MyModel_Under(tf.keras.Model):
    def __init__(self):
        super(MyModel_Under, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(32, activation = 'relu')
        # weight: 32 X 64 matrix
        self.dense2 = tf.keras.layers.Dense(64, activation = 'relu')
        # weight: 64 X 128 matrix
        self.dense3 = tf.keras.layers.Dense(128, activation = 'relu')
        # weight: 128 X 256 matrix
        self.dense4 = tf.keras.layers.Dense(256, activation = 'relu')
        self.dense5 = tf.keras.layers.Dense(10, activation = 'softmax')
        
    def call(self, x, training = False, mask = None):
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        
        return self.dense5(x)

In [11]:
# loading data
fashion_mnist = tf.keras.datasets.fashion_mnist # 28 X 28 (10 classes)

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32).prefetch(2048)
test_ds = tf.data.Dataset.from_tensor_slices(
    (x_test, y_test)).batch(32).prefetch(2048)

In [12]:
# training
model3 = MyModel_Under()
model3.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])
model3.fit(train_ds, validation_data = test_ds, epochs = EPOCHS)

Epoch 1/100
1875/1875 [==============================] - 10s 5ms/step - loss: 0.5125 - accuracy: 0.8124 - val_loss: 0.4469 - val_accuracy: 0.8372
Epoch 2/100
1875/1875 [==============================] - 10s 6ms/step - loss: 0.3886 - accuracy: 0.8557 - val_loss: 0.4075 - val_accuracy: 0.8506
Epoch 3/100
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3547 - accuracy: 0.8698 - val_loss: 0.3939 - val_accuracy: 0.8611
Epoch 4/100
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3342 - accuracy: 0.8767 - val_loss: 0.3711 - val_accuracy: 0.8652
Epoch 5/100
1875/1875 [==============================] - 10s 5ms/step - loss: 0.3168 - accuracy: 0.8820 - val_loss: 0.3955 - val_accuracy: 0.8554
Epoch 6/100
1875/1875 [==============================] - 10s 6ms/step - loss: 0.3056 - accuracy: 0.8857 - val_loss: 0.3485 - val_accuracy: 0.8731
Epoch 7/100
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2939 - accuracy: 0.8907 - val_loss: 0.3513 - va

Epoch 57/100
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1380 - accuracy: 0.9461 - val_loss: 0.5819 - val_accuracy: 0.8815
Epoch 58/100
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1365 - accuracy: 0.9469 - val_loss: 0.5725 - val_accuracy: 0.8780
Epoch 59/100
1875/1875 [==============================] - 10s 6ms/step - loss: 0.1365 - accuracy: 0.9469 - val_loss: 0.5862 - val_accuracy: 0.8809
Epoch 60/100
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1348 - accuracy: 0.9474 - val_loss: 0.6527 - val_accuracy: 0.8816
Epoch 61/100
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1366 - accuracy: 0.9478 - val_loss: 0.6106 - val_accuracy: 0.8814
Epoch 62/100
1875/1875 [==============================] - 11s 6ms/step - loss: 0.1330 - accuracy: 0.9484 - val_loss: 0.5902 - val_accuracy: 0.8841
Epoch 63/100
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1347 - accuracy: 0.9477 - val_loss: 0

The modified model's performance might be worse than the original one. However, our objective is to maximize its performance given computing resources (given the model's complexity).

In [14]:
# DenseNet
class MyModel_Under(tf.keras.Model):
    def __init__(self):
        super(MyModel_Under, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(32, use_bias = False)
        self.batch1 = tf.keras.layers.BatchNormalization()
        
        # preactivation (Recall DenseNet!) -> less resource usage!
        #                                     less number of operations!
        # weight: 32 X 32 matrix
        self.batch2 = tf.keras.layers.BatchNormalization()
        self.dense2 = tf.keras.layers.Dense(32, use_bias = False)
        
        # weight: 64 X 64 matrix (See the concat below call function.)
        self.batch3 = tf.keras.layers.BatchNormalization()
        self.dense3 = tf.keras.layers.Dense(64, use_bias = False)
        
        # weight: 128 X 128 matrix
        self.batch4 = tf.keras.layers.BatchNormalization()
        self.dense4 = tf.keras.layers.Dense(128, use_bias = False)
        
        self.dense5 = tf.keras.layers.Dense(10, activation = 'softmax')
        
    def call(self, x, training = False, mask = None):
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.batch1(x, training)
        x = tf.nn.relu(x)
        
        # preactivation
        h = self.batch2(x, training)
        h = tf.nn.relu(h)
        h = self.dense2(h)
        x = tf.concat([x, h], axis = -1)

        h = self.batch3(x, training)
        h = tf.nn.relu(h)
        h = self.dense3(h)
        x = tf.concat([x, h], axis = -1)

        h = self.batch4(x, training)
        h = tf.nn.relu(h)
        h = self.dense4(h)
        x = tf.concat([x, h], axis = -1)
        
        return self.dense5(x)

In [15]:
# training
model4 = MyModel_Under()
model4.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])
model4.fit(train_ds, validation_data = test_ds, epochs = EPOCHS)

Epoch 1/100
1875/1875 [==============================] - 24s 12ms/step - loss: 0.5115 - accuracy: 0.8174 - val_loss: 0.5607 - val_accuracy: 0.7876
Epoch 2/100
1875/1875 [==============================] - 24s 13ms/step - loss: 0.4035 - accuracy: 0.8532 - val_loss: 0.4223 - val_accuracy: 0.8508
Epoch 3/100
1875/1875 [==============================] - 24s 13ms/step - loss: 0.3740 - accuracy: 0.8617 - val_loss: 0.3788 - val_accuracy: 0.8646
Epoch 4/100
1875/1875 [==============================] - 24s 13ms/step - loss: 0.3494 - accuracy: 0.8719 - val_loss: 0.3951 - val_accuracy: 0.8605
Epoch 5/100
1875/1875 [==============================] - 24s 13ms/step - loss: 0.3337 - accuracy: 0.8769 - val_loss: 0.3583 - val_accuracy: 0.8699
Epoch 6/100
1875/1875 [==============================] - 24s 13ms/step - loss: 0.3229 - accuracy: 0.8814 - val_loss: 0.4083 - val_accuracy: 0.8515
Epoch 7/100
1875/1875 [==============================] - 24s 13ms/step - loss: 0.3142 - accuracy: 0.8845 - val_loss: 0

1875/1875 [==============================] - 24s 13ms/step - loss: 0.1866 - accuracy: 0.9295 - val_loss: 0.4033 - val_accuracy: 0.8753
Epoch 57/100
1875/1875 [==============================] - 23s 12ms/step - loss: 0.1851 - accuracy: 0.9300 - val_loss: 0.4024 - val_accuracy: 0.8822
Epoch 58/100
1875/1875 [==============================] - 22s 12ms/step - loss: 0.1834 - accuracy: 0.9307 - val_loss: 0.4217 - val_accuracy: 0.8807
Epoch 59/100
1875/1875 [==============================] - 22s 12ms/step - loss: 0.1837 - accuracy: 0.9312 - val_loss: 0.4014 - val_accuracy: 0.8813
Epoch 60/100
1875/1875 [==============================] - 22s 12ms/step - loss: 0.1821 - accuracy: 0.9313 - val_loss: 0.4013 - val_accuracy: 0.8798
Epoch 61/100
1875/1875 [==============================] - 24s 13ms/step - loss: 0.1810 - accuracy: 0.9319 - val_loss: 0.4282 - val_accuracy: 0.8740
Epoch 62/100
1875/1875 [==============================] - 24s 13ms/step - loss: 0.1828 - accuracy: 0.9313 - val_loss: 0.3989 

# Imbalanced Data Problem
---

In [31]:
class MyModel_Aug(tf.keras.Model):
    def __init__(self):
        super(MyModel_Aug, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(1024, activation = 'relu')
        self.dense2 = tf.keras.layers.Dense(1, activation = 'sigmoid')
        
    def call(self, x, training = False, mask = None):
        x = self.flatten(x)
        x = self.dense1(x)
        
        return self.dense2(x)

In [32]:
# loading data
cifar10 = tf.keras.datasets.cifar10 # 32 X 32 X 3 video (10 classes)

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

# imbalanced small dataset
import random

# training set
x_train_small = list() # Its elements are batches.
y_train_small = list()

for x, y in zip(x_train, y_train):
    # Use only 10% data if labels are 0. -> Intentionally, generate small size class.
    # Use 100% data if labels are 1.
    if (y == 0 and random.randint(0, 100) < 10) or y == 1:
        x_train_small.append(x[:]) # flattenning! (Our network is shallow.)
        y_train_small.append(y)
        
# test set
x_test_small = list()
y_test_small = list()

for x, y in zip(x_test, y_test):
    # The test set must be extracted fairly.
    if y == 0 or y == 1:
        x_test_small.append(x[:])
        y_test_small.append(y)
        
# Convert the manipulated data into NumPy arrays..
x_train = np.stack(x_train_small, axis = 0)
y_train = np.stack(y_train_small, axis = 0)
x_test = np.stack(x_test_small, axis = 0)
y_test = np.stack(y_test_small, axis = 0)

train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32).prefetch(2048)
test_ds = tf.data.Dataset.from_tensor_slices(
    (x_test, y_test)).batch(32).prefetch(2048)

In [33]:
# training
model5 = MyModel_Aug()
model5.compile(optimizer = 'adam',
               loss = 'binary_crossentropy',
               # Precision and Recall are useful to detect the problem
               # caused by the model which ignores imbalanced data.
               metrics = ['accuracy',
                          tf.keras.metrics.Precision(name = 'precision'),
                          tf.keras.metrics.Recall(name = 'recall')])
model5.fit(train_ds, validation_data = test_ds, epochs = EPOCHS)

# Note that the validation precision is almost much less thant
# that of training,
# which indicates an imblanced data problem.
# (invariant training loss over epochs.. no training improvement)

Epoch 1/100
173/173 [==============================] - 2s 9ms/step - loss: 0.4258 - accuracy: 0.9028 - precision: 0.9152 - recall: 0.9840 - val_loss: 0.8537 - val_accuracy: 0.5850 - val_precision: 0.5473 - val_recall: 0.9830
Epoch 2/100
173/173 [==============================] - 1s 8ms/step - loss: 0.2782 - accuracy: 0.9059 - precision: 0.9236 - recall: 0.9770 - val_loss: 0.7234 - val_accuracy: 0.6140 - val_precision: 0.5654 - val_recall: 0.9850
Epoch 3/100
173/173 [==============================] - 1s 8ms/step - loss: 0.2297 - accuracy: 0.9175 - precision: 0.9285 - recall: 0.9848 - val_loss: 1.2338 - val_accuracy: 0.5445 - val_precision: 0.5234 - val_recall: 0.9940
Epoch 4/100
173/173 [==============================] - 1s 9ms/step - loss: 0.2325 - accuracy: 0.9173 - precision: 0.9316 - recall: 0.9808 - val_loss: 0.8245 - val_accuracy: 0.6380 - val_precision: 0.5813 - val_recall: 0.9870
Epoch 5/100
173/173 [==============================] - 2s 9ms/step - loss: 0.2102 - accuracy: 0.9271

173/173 [==============================] - 1s 8ms/step - loss: 0.0661 - accuracy: 0.9750 - precision: 0.9825 - recall: 0.9900 - val_loss: 0.8275 - val_accuracy: 0.7685 - val_precision: 0.6887 - val_recall: 0.9800
Epoch 38/100
173/173 [==============================] - 1s 9ms/step - loss: 0.0786 - accuracy: 0.9708 - precision: 0.9800 - recall: 0.9880 - val_loss: 0.5605 - val_accuracy: 0.8295 - val_precision: 0.7580 - val_recall: 0.9680
Epoch 39/100
173/173 [==============================] - 2s 9ms/step - loss: 0.0635 - accuracy: 0.9772 - precision: 0.9837 - recall: 0.9912 - val_loss: 1.0375 - val_accuracy: 0.7410 - val_precision: 0.6607 - val_recall: 0.9910
Epoch 40/100
173/173 [==============================] - 1s 8ms/step - loss: 0.0653 - accuracy: 0.9763 - precision: 0.9829 - recall: 0.9910 - val_loss: 1.0126 - val_accuracy: 0.7395 - val_precision: 0.6600 - val_recall: 0.9880
Epoch 41/100
173/173 [==============================] - 1s 8ms/step - loss: 0.0601 - accuracy: 0.9768 - preci

173/173 [==============================] - 1s 9ms/step - loss: 0.0258 - accuracy: 0.9906 - precision: 0.9948 - recall: 0.9948 - val_loss: 0.8942 - val_accuracy: 0.8150 - val_precision: 0.7442 - val_recall: 0.9600
Epoch 74/100
173/173 [==============================] - 1s 8ms/step - loss: 0.0297 - accuracy: 0.9891 - precision: 0.9940 - recall: 0.9940 - val_loss: 1.2201 - val_accuracy: 0.7730 - val_precision: 0.6931 - val_recall: 0.9800
Epoch 75/100
173/173 [==============================] - 1s 8ms/step - loss: 0.0181 - accuracy: 0.9937 - precision: 0.9954 - recall: 0.9976 - val_loss: 0.8676 - val_accuracy: 0.8260 - val_precision: 0.7527 - val_recall: 0.9710
Epoch 76/100
173/173 [==============================] - 1s 9ms/step - loss: 0.0405 - accuracy: 0.9862 - precision: 0.9930 - recall: 0.9918 - val_loss: 0.6761 - val_accuracy: 0.8295 - val_precision: 0.7998 - val_recall: 0.8790
Epoch 77/100
173/173 [==============================] - 2s 9ms/step - loss: 0.0311 - accuracy: 0.9882 - preci

In [24]:
!pip install imblearn

In [34]:
# applying BorderlineSMOTE
from imblearn.over_sampling import BorderlineSMOTE

# instantiating SMOTE
smote = BorderlineSMOTE()

# Convert the shape of data. (from 4-dimensional to 2-dimensional)
# Note that our model is fully connected layers not CNN.
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1] * x_train.shape[2] * x_train.shape[3])).astype(np.float32)
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1] * x_test.shape[2] * x_test.shape[3])).astype(np.float32)

x_train, y_train = smote.fit_resample(x_train, y_train)

train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32).prefetch(2048)
test_ds = tf.data.Dataset.from_tensor_slices(
    (x_test, y_test)).batch(32).prefetch(2048)

In [35]:
# training
model6 = MyModel_Aug()
model6.compile(optimizer = 'adam',
               loss = 'binary_crossentropy',
               # Precision and Recall are useful to detect the problem
               # caused by the model which ignores imbalanced data.
               metrics = ['accuracy',
                          tf.keras.metrics.Precision(name = 'precision'),
                          tf.keras.metrics.Recall(name = 'recall')])
model6.fit(train_ds, validation_data = test_ds, epochs = EPOCHS)

Epoch 1/100
313/313 [==============================] - 3s 8ms/step - loss: 0.5616 - accuracy: 0.7491 - precision: 0.7601 - recall: 0.7280 - val_loss: 0.4419 - val_accuracy: 0.8150 - val_precision: 0.8387 - val_recall: 0.7800
Epoch 2/100
313/313 [==============================] - 2s 7ms/step - loss: 0.4233 - accuracy: 0.8129 - precision: 0.8287 - recall: 0.7888 - val_loss: 0.5645 - val_accuracy: 0.7565 - val_precision: 0.6942 - val_recall: 0.9170
Epoch 3/100
313/313 [==============================] - 2s 8ms/step - loss: 0.3333 - accuracy: 0.8623 - precision: 0.8863 - recall: 0.8312 - val_loss: 0.5580 - val_accuracy: 0.7555 - val_precision: 0.9237 - val_recall: 0.5570
Epoch 4/100
313/313 [==============================] - 2s 8ms/step - loss: 0.2707 - accuracy: 0.8955 - precision: 0.9230 - recall: 0.8630 - val_loss: 0.4739 - val_accuracy: 0.8180 - val_precision: 0.7686 - val_recall: 0.9100
Epoch 5/100
313/313 [==============================] - 2s 8ms/step - loss: 0.2303 - accuracy: 0.9147

313/313 [==============================] - 2s 8ms/step - loss: 0.0217 - accuracy: 0.9932 - precision: 0.9946 - recall: 0.9918 - val_loss: 0.8696 - val_accuracy: 0.8650 - val_precision: 0.8219 - val_recall: 0.9320
Epoch 38/100
313/313 [==============================] - 2s 8ms/step - loss: 0.0177 - accuracy: 0.9939 - precision: 0.9960 - recall: 0.9918 - val_loss: 0.9625 - val_accuracy: 0.8405 - val_precision: 0.7766 - val_recall: 0.9560
Epoch 39/100
313/313 [==============================] - 2s 8ms/step - loss: 0.0542 - accuracy: 0.9806 - precision: 0.9835 - recall: 0.9776 - val_loss: 1.1798 - val_accuracy: 0.8235 - val_precision: 0.7506 - val_recall: 0.9690
Epoch 40/100
313/313 [==============================] - 2s 8ms/step - loss: 0.0299 - accuracy: 0.9900 - precision: 0.9922 - recall: 0.9878 - val_loss: 1.0702 - val_accuracy: 0.8390 - val_precision: 0.7699 - val_recall: 0.9670
Epoch 41/100
313/313 [==============================] - 2s 8ms/step - loss: 0.0199 - accuracy: 0.9941 - preci

313/313 [==============================] - 2s 8ms/step - loss: 0.0154 - accuracy: 0.9953 - precision: 0.9958 - recall: 0.9948 - val_loss: 1.1977 - val_accuracy: 0.8365 - val_precision: 0.7924 - val_recall: 0.9120
Epoch 74/100
313/313 [==============================] - 2s 8ms/step - loss: 0.0549 - accuracy: 0.9845 - precision: 0.9869 - recall: 0.9820 - val_loss: 1.5090 - val_accuracy: 0.8190 - val_precision: 0.7409 - val_recall: 0.9810
Epoch 75/100
313/313 [==============================] - 2s 8ms/step - loss: 0.0126 - accuracy: 0.9964 - precision: 0.9970 - recall: 0.9958 - val_loss: 1.1917 - val_accuracy: 0.8455 - val_precision: 0.7901 - val_recall: 0.9410
Epoch 76/100
313/313 [==============================] - 2s 8ms/step - loss: 0.0037 - accuracy: 0.9990 - precision: 0.9996 - recall: 0.9984 - val_loss: 1.3827 - val_accuracy: 0.8365 - val_precision: 0.7652 - val_recall: 0.9710
Epoch 77/100
313/313 [==============================] - 2s 8ms/step - loss: 0.0038 - accuracy: 0.9987 - preci

It is not appropriate to apply SMOTE to video data.<br>
It is better to apply SMOTE after we extract features from video data, which will yield better performance models.